<a href="https://colab.research.google.com/github/jameschapman19/education-challenge/blob/main/CCA_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An interactive introduction to Multiview Learning with Canonical Correlation Analysis and Partial Least Squares using cca-zoo


## Table of Contents
1. [Objective of the Tutorial](#obj)
2. [Set-up](#set-up)
3. [Introduction to multiview machine learning](#mv)
  *   [Machine Learning Framework](#mlframework)
  *   [Latent Variable Models](#lv)
4. [Simulating Data](#datagen)
5. [Overfitting and Sample Size](#samplesize)
6. [Ridge Regularisation: From CCA to PLS](#ridge)
7. [Lasso Regularisation](#lasso)
  *   [Sparse Partial Least Squares](#spls)
  *   [Sparse Canonical Correlation Analysis](#scca)
8. [Application: a neuroimaging case study](#haxby)
  *   [Regularised CCA](#rccahaxby)
  *   [Sparse Partial Least Squares](#splshaxby)
  *   [Sparse Canonical Correlation Analysis](#sccahaxby)
9. [Conclusion](#conclusion)
10. [References](#references)

## Objective <a name="Set-up"></a>
With the large multiview and multimodal datasets available in medical imaging and healthcare informatics, there is a strong need for multivariate statistical and machine learning methods which can capture relationships across different views of data.

Canonical correlation analysis and partial least squares are arguably the two most commonly used methods in modelling two-view data with latent variable models. Extensions to these methods have included Generalized CCA (and Multiset CCA) which can capture relationships between more than 2 views and various forms of regularised CCA and PLS.

This tutorial has three main aims 

*   to give an interactive and visual introduction to CCA and PLS and to demonstrate the effect and benefits of using regularised versions of these classical methods
*   to demonstrate how we might use these methods in a neuroimaging example
*   to introduce cca-zoo, an open source python package containing implementations of this family of algorithms. The source code is available at https://github.com/jameschapman19/cca_zoo and documentation at https://cca-zoo.readthedocs.io/en/latest/index.html

## Set-up <a name="set-up"></a>
This tutorial depends on the package cca-zoo and nilearn for a simulated neuroimaging example.

In order to make use of the interactive elements, the notebook should be run in a google colab instance.

### Installation and Imports

In [1]:
# @markdown Execute this cell to install cca-zoo and nilearn
!pip install cca-zoo --upgrade
!pip install nilearn

     |████████████████████████████████| 68 kB 4.8 MB/s 
     |████████████████████████████████| 160 kB 31.6 MB/s 
     |████████████████████████████████| 2.1 MB 81.4 MB/s 
     |████████████████████████████████| 28.5 MB 50 kB/s 
     |████████████████████████████████| 154 kB 86.6 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 10.0 MB 9.9 MB/s 


In [2]:
# @markdown Execute this cell to import modules
import ipywidgets as widgets
import seaborn as sns
from cca_zoo.models import rCCA, PMD, SCCA
from cca_zoo.data import generate_covariance_data
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from nilearn import datasets
from nilearn.image import index_img
from nilearn.input_data import NiftiMasker
from nilearn.plotting import view_img
from sklearn.preprocessing import OneHotEncoder
from nilearn.plotting import plot_stat_map, show
from sklearn.model_selection import train_test_split
np.random.seed(42)
sns.set(font_scale=1)

/usr/local/lib/python3.7/dist-packages/nilearn/datasets/__init__.py:96: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


### Plotting Helpers
We will use a number of standard figures to visualise the results of models used throughout this notebook

In [3]:
# @markdown Execute this cell to access plotting functions
def plot_latent_train_test(train_scores, test_scores, title='Projections of data in latent space'):
    train_data = pd.DataFrame(
        {'phase': np.asarray(['train'] * train_scores[0].shape[0]).astype(str)})
    x_vars=[f'View 1 dimension {f}' for f in range(1,train_scores[0].shape[1]+1)]
    y_vars=[f'View 2 dimension {f}' for f in range(1,train_scores[1].shape[1]+1)]
    train_data[x_vars] = train_scores[0]
    train_data[y_vars] = train_scores[1]
    test_data = pd.DataFrame(
        {'phase': np.asarray(['test'] * test_scores[0].shape[0]).astype(str)})
    test_data[x_vars] = test_scores[0]
    test_data[y_vars] = test_scores[1]
    data = pd.concat([train_data, test_data], axis=0)
    cca_pp = sns.pairplot(data, hue='phase',x_vars=x_vars,y_vars=y_vars, corner=True)
    cca_pp.fig.set_size_inches(10,5)
    if title:
      cca_pp.fig.suptitle(title)
    return cca_pp

def plot_train_test_corrs(train_scores, test_scores, title='Train vs Test correlations'):
    latent_dims=train_scores[0].shape[1]
    train_corrs=np.diag(np.corrcoef(train_scores[0],train_scores[1],rowvar=False)[:latent_dims,latent_dims:])
    test_corrs=np.diag(np.corrcoef(test_scores[0],test_scores[1],rowvar=False)[:latent_dims,latent_dims:])
    train_corr_data=pd.DataFrame({'correlation':train_corrs,'dimension':np.arange(latent_dims)+1,'phase': np.asarray(['train'] * latent_dims).astype(str)})
    test_corr_data=pd.DataFrame({'correlation':test_corrs,'dimension':np.arange(latent_dims)+1,'phase': np.asarray(['test'] * latent_dims).astype(str)})
    corr_data = pd.concat([train_corr_data, test_corr_data], axis=0)
    # setting the dimensions of the plot
    fig2, ax = plt.subplots()
    cca_bp=sns.barplot(x="dimension", y="correlation", hue="phase", data=corr_data,ax=ax)
    cca_bp.set_ylim(bottom=-1, top=1)
    if title:
      cca_bp.set_title(title)
    return cca_bp

def plot_true_weights_coloured(ax, weights, true_weights=None, title=''):
    if true_weights is None:
      true_weights=np.ones(len(weights))
    ind = np.arange(len(true_weights))
    mask = np.squeeze(true_weights == 0)
    ax.scatter(ind[~mask], weights[~mask], c='b',label='Non-Zero')
    ax.scatter(ind[mask], weights[mask], c='r',label='Zero')
    ax.set_title(title)

def plot_model_weights(wx,wy,tx=None,ty=None, title='Model weights vs. True weights'):
    if tx is None and ty is None:
      fig,axs=plt.subplots(1,2)
      plot_true_weights_coloured(axs[0],wx,title='model view 1 weights')
      plot_true_weights_coloured(axs[1],wy,title='model view 2 weights')
      axs[1].legend()
    else:
      fig,axs=plt.subplots(2,2,sharex=True,sharey=True)
      plot_true_weights_coloured(axs[0,0],tx,tx,title='true view 1 weights')
      plot_true_weights_coloured(axs[0,1],ty,ty,title='true view 2 weights')
      plot_true_weights_coloured(axs[1,0],wx,tx,title='model view 1 weights')
      plot_true_weights_coloured(axs[1,1],wy,ty,title='model view 2 weights')
      axs[0,1].legend()
    plt.tight_layout()
    return fig

## Introduction to Multivariate Machine Learning <a name="mv"></a>

### Latent Variable Models <a name="lv"></a>
Latent variable models assume that the two views of data are derived from some shared latent (unobserved) variables. For neuroimaging and behaviour studies we might expect an underlying health condition to influence both an MRI scan of a patient and data relating to their behaviour.

PLS and CCA models work by projecting the observed data from each view into latent variables for each view that are highly correlated. These can be interpeted as estimates of the true latent variable. Often in neuroimaging studies the biggest components can be shown to be related to variables like Age, Scanner Type and Gender.

![Latent Variable Model of Brain-Behaviour data](https://raw.githubusercontent.com/jameschapman19/education-challenge/main/image.png)

### Optimisation Problems
Partial Least Squares and Canonical Correlation Analysis share the same objective function (maximising covariance between the projections of each view). The difference is that PLS constrains the variance of the weights (their 2 norm) whereas CCA constrains the variance of the projections.

#### Partial Least Squares
$w_{opt}=\underset{w}{\mathrm{argmax}}\{ w_1^TX_1^TX_2w_2  \}$

$\text{subject to:}$

$\|w_1\|_2=1$,
$\|w_2\|_2=1$

#### Canonical Correlation Analysis
$w_{opt}=\underset{w}{\mathrm{argmax}}\{ w_1^TX_1^TX_2w_2  \}$

$\text{subject to:}$

$\|X_1w_1\|_2=1$

$\|X_2w_2\|_2=1$


The result of these optimisations is that PLS is much more biased towards the largest principal components in the data whereas CCA is sensitive to high correlations, even if they only explain small ammounts of the data.

Also notice that in the special case where the covariance matrices for each view ($X_1^TX_1$ and $X_2^TX_2$) are identity matrices $I$, the CCA and PLS problems are identical (since $\|X_iw_i\|_2=w_i^TX_i^TX_iw_i=w_i^Tw_i$).

#### Relationship to PCA
It is interesting to compare these objectives to the more familiar principal components analysis. In PCA, we find weights that project the data in directions that maximise variance. If our two views are the same then PLS is exactly the same model as PCA!

$w_{opt}=\underset{w}{\mathrm{argmax}}\{ w^TX^TXw  \}$

$\text{subject to:}$

$\|w\|_2=1$,

### Machine Learning Framework <a name="mlframework"></a>

In a machine learning framework, we are primarily interested in the generalization of a model to data not used in the training process. In this tutorial we will generate simulated data which we will split into train and test data to demonstrate how different models generalize.

![Framework](https://raw.githubusercontent.com/jameschapman19/education-challenge/main/image2.png)

### Simulated Data <a name="datagen"></a>
We will use a joint multivariate normal distribution to generate two views of data: view 1 and view 2. This method, described in detail by Helmer et al [1], allows us to control:

*   Number of training samples
*   Number of test samples
*   Number of features in view 1
*   Number of features in view 2
*   Level of sparsity in view 1 (fraction of variables from view 1 involved in correlation)
*   Level of sparsity in view 2 (fraction of variables from view 2 involved in correlation)

As well as the strength of the population correlation and the covariance structure within each view. We assume that the underlying correlation is perfect (1) and that the within-view variance is identity.

This data generation process is also equivalent to a latent variable model where view 1 and view 2 are conditionally independent given some latent variable Z.

In [4]:
# @markdown Execute this cell to choose data parameters! You can see the projections of the generated data using the true weights. These should have almost perfect correlations in both the training and test data.
style = {'description_width': 'initial'}

N_train= widgets.IntSlider(value=100,min=10,max=500,description='Train Samples',style=style,continuous_update=False)
N_test= widgets.IntSlider(value=100,min=10,max=500,description='Test Samples',style=style,continuous_update=False)
X_features=widgets.IntSlider(value=60,min=10,max=500,description='View 1 features',style=style,continuous_update=False)
Y_features=widgets.IntSlider(value=60,min=10,max=500,description='View 2 features',style=style,continuous_update=False)
view_1_sparsity=widgets.FloatSlider(value=0.5,min=0,max=1,description='View 1 Sparsity',style=style,continuous_update=False)
view_2_sparsity=widgets.FloatSlider(value=0.5,min=0,max=1,description='View 2 Sparsity',style=style,continuous_update=False)

def generate_data(N_train,N_test,X_features,Y_features, view_1_sparsity, view_2_sparsity):
    (X,Y),(tx,ty)=generate_covariance_data(N_train+N_test,view_features=[X_features,Y_features],latent_dims=1,correlation=1, view_sparsity=[view_1_sparsity, view_2_sparsity])
    X_tr,X_te,Y_tr,Y_te=train_test_split(X,Y,train_size=N_train)
    plot_latent_train_test(np.stack((X_tr@tx,Y_tr@ty)), np.stack((X_te@tx,Y_te@ty)), title='Projections of data in latent space using true model weights')
    return (X_tr,X_te,Y_tr,Y_te,tx,ty)

out=widgets.interactive(generate_data, N_train=N_train,N_test=N_test,X_features=X_features,Y_features=Y_features,view_1_sparsity=view_1_sparsity,view_2_sparsity=view_2_sparsity)
display(out)

interactive(children=(IntSlider(value=100, continuous_update=False, description='Train Samples', max=500, min=…

## Overfitting and Sample Size <a name="samplesize"></a>
When the sample size is smaller than the number of features, CCA will overfit the data (you can try this by changing the data parameters). Visually, CCA overfitting looks like all the training samples are projected with perfect correlation but the test samples are almost completely uncorrelated. The intuition behind this is that there will be at least one spurious relationship when the data are not full rank.

PLS on the other hand is always well-posed even when the number of samples is less than the number of features. However since PLS maximises covariance rather than correlation, the correlation of the latent variables in the training data is typically much lower. 

In [5]:
# @markdown Execute this cell to toggle between CCA and PLS models (you may need to toggle when you change the data!)
X_tr,X_te,Y_tr,Y_te, tx,ty=out.result
tog=widgets.ToggleButtons(
    options=['CCA', 'PLS'],
    description='Model:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)
def interactive_cca(tog):
  if tog=='CCA':
    rcca=rCCA(latent_dims=1,c=1e-9).fit([X_tr,Y_tr])
  elif tog=='PLS':
    rcca=rCCA(latent_dims=1,c=1).fit([X_tr,Y_tr])
  test_scores=rcca.transform([X_te,Y_te])
  plot_latent_train_test(rcca.scores,test_scores)

plot_widget=widgets.interactive(interactive_cca,tog=tog)
display(plot_widget)

interactive(children=(ToggleButtons(description='Model:', options=('CCA', 'PLS'), value='CCA'), Output()), _do…

## Ridge Regularised CCA: from CCA to PLS <a name="ridge"></a>

Given that PLS has an effect that is similar to ridge regularisation, a natural extension is to mix CCA and PLS to control this regularisation effect. Vinod proposed the Canonical Ridge model [2] which combines the constraints from the two models with mixing parameters $c_1$ and $c_2$ which make the model more or less CCA-like or PLS-like.

$w_{opt}=\underset{w}{\mathrm{argmax}}\{ w_1^TX_1^TX_2w_2  \}$

$\text{subject to:}$

$(1-c_1)\|X_1w_1\|_2+c_1\|w_1\|_2=1$

$(1-c_2)\|X_2w_2\|_2+c_2\|w_2\|_2=1$

In the next interactive widget, you can vary $c_1$ and $c_2$ yourself. The plots are provided to help visualise:

*   The projections into latent space
*   The train and test correlations
*   The model weights vs. the true weights

In [6]:
# @markdown Execute this cell to change model regularisation (there's a bit of a lag as the model needs to fit in the background!)
X_tr,X_te,Y_tr,Y_te, tx,ty=out.result
style = {'description_width': 'initial'}
cx=widgets.FloatLogSlider(base=10,value=-1, min=-10, max=0,description='cx',readout=True,readout_format='.9f',style=style,continuous_update=False)
cy=widgets.FloatLogSlider(base=10,value=-1, min=-10, max=0,description='cy',readout=True,readout_format='.9f',style=style,continuous_update=False)
def interactive_cca(cx,cy):
    rcca=rCCA(latent_dims=1,c=[cx,cy]).fit([X_tr,Y_tr])
    test_scores=rcca.transform(X_te,Y_te)
    plot_latent_train_test(rcca.scores,test_scores)
    plot_train_test_corrs(rcca.scores,test_scores)
    plot_model_weights(rcca.weights[0],rcca.weights[1],tx,ty, title='Model weights vs. True weights')

plot_widget=widgets.interactive(interactive_cca, cx=cx,cy=cy)
display(plot_widget)

interactive(children=(FloatLogSlider(value=1e-10, continuous_update=False, description='cx', max=0.0, min=-10.…

As we increase $c_1$ and $c_2$, the correlation of the training data decreases (and the covariance increases). 

There is often a 'sweet spot' somewhere between CCA and PLS where the test correlation is maximised. You can see the train and test correlations in the bar plot.

We now also display the learnt model weights alongside the true weights.

## Lasso Regularisation <a name="lasso"></a>
Just as the canonical ridge is analagous to ridge regularised regression, we can also use lasso regularisation in PLS and CCA models. Lasso regularisation is helpful in biomedical applications as the effects we are looking for are often parsimonious (only a small number of the features contain a signal) and the natural feature selection mechanism helps interpretability.

In this section, we introduce Sparse Partial Least Squares and Sparse Canonical Correlation Analysis.

### Sparse Partial Least Squares <a name="spls"></a>
By adding an additional constraint on the weights, Witten demonstrated a sparse partial least squares model [3] able to recover signals that were only present in a fraction of the variables. Since this method adapts the PLS optimization problem, it also tends to show the same ridge regularisation properties.

$w_{opt}=\underset{w}{\mathrm{argmax}}\{ w_1^TX_1^TX_2w_2  \}$

$\text{subject to:}$

$\|w_1\|_2=1$,
$\|w_1\|_2=1$

$\|w_1\|_1\leq c_1$,
$\|w_2\|_1\leq c_2$

reducing the value of $c_1$ and $c_2$ leads to a sparser solution with more zero weights 

Have a look for yourself!

In [7]:
# @markdown Execute this cell to change model regularisation
X_tr,X_te,Y_tr,Y_te, tx,ty=out.result
style = {'description_width': 'initial'}
c1=widgets.FloatSlider(value=3, min=1, max=np.sqrt(X_tr.shape[1]),description='c1',readout=True,readout_format='.5f',style=style,continuous_update=False)
c2=widgets.FloatSlider(value=3, min=1, max=np.sqrt(Y_tr.shape[1]),description='c2',readout=True,readout_format='.5f',style=style,continuous_update=False)
def interactive_cca(c1,c2):
    spls=PMD(latent_dims=1,c=[c1,c2]).fit([X_tr,Y_tr])
    test_scores=spls.transform([X_te,Y_te])
    plot_latent_train_test(spls.scores,test_scores)
    plot_train_test_corrs(spls.scores,test_scores)
    plot_model_weights(spls.weights[0],spls.weights[1],tx,ty, title='Model weights vs. True weights')

plot_widget=widgets.interactive(interactive_cca, c1=c1,c2=c2)
display(plot_widget)

interactive(children=(FloatSlider(value=3.0, continuous_update=False, description='c1', max=7.745966692414834,…

### Sparse Canonical Correlation Analysis <a name="scca"></a>
There have been a number of attempts to construct sparse CCA models. One of the most succesful is Mai's variant [4] which uses a lasso penalty with a form much like lasso regression [7].


$w_{opt}=\underset{w}{\mathrm{argmax}}\{ w_1^TX_1^TX_2w_2 - c_1\|w_1\|_1- c_2\|w_2\|_1\}$

$\text{subject to:}$

$w_1^TX_1^TX_1w_1=1$

$w_2^TX_2^TX_2w_2=1$

This is a slightly different form to the sparse PLS model. Since the 1-norm of the weights penalizes the objective rather than being a constraint, a higher value of $c_1$ and $c_2$ leads to a more sparse (more weights equal to zero) solution.

Have a go for yourself!

In [8]:
# @markdown Execute this cell to change model regularisation
X_tr,X_te,Y_tr,Y_te, tx,ty=out.result
style = {'description_width': 'initial'}
c1=widgets.FloatLogSlider(value=0.005,base=10,min=-10, max=0,description='c1',readout=True,readout_format='.9f',style=style,continuous_update=False)
c2=widgets.FloatLogSlider(value=0.005,base=10,min=-10, max=0,description='c2',readout=True,readout_format='.9f',style=style,continuous_update=False)
def interactive_cca(c1,c2):
    scca=SCCA(latent_dims=1,c=[c1,c2]).fit([X_tr,Y_tr])
    test_scores=scca.transform([X_te,Y_te])
    plot_latent_train_test(scca.scores,test_scores)
    plot_train_test_corrs(scca.scores,test_scores)
    plot_model_weights(scca.weights[0],scca.weights[1],tx,ty, title='Model weights vs. True weights')

plot_widget=widgets.interactive(interactive_cca, c1=c1,c2=c2)
display(plot_widget)

interactive(children=(FloatLogSlider(value=0.005, continuous_update=False, description='c1', max=0.0, min=-10.…

## Apply what we have learnt: a neuroimaging example <a name="haxby"></a>

This part leans heavily on the nilearn [5,6] tutorial in:
https://nilearn.github.io/auto_examples/02_decoding/plot_haxby_different_estimators.html#sphx-glr-auto-examples-02-decoding-plot-haxby-different-estimators-py.

The Haxby dataset a well known neuroscience dataset which contains functional magnetic resonance images (fMRI) from various subjects while they observe different tasks (e.g. house, cat). 

We will apply our multivariate models to this data.

We take masked versions of the fMRI data as view 1. As the other view we will take one hot encoded matrix of the 12 task labels. We will also add 12 random columns to view 2 which we hope our model will learn to ignore since there should be no correlated signal with the fMRI data.

![Hax](https://raw.githubusercontent.com/jameschapman19/education-challenge/main/image3.png)

In [9]:
# @markdown Execute this cell to fetch and preprocess the Haxby Dataset!
haxby_dataset = datasets.fetch_haxby()
mask_filename = haxby_dataset.mask_vt[0]
fmri_filename = haxby_dataset.func[0]
behavioural = pd.read_csv(haxby_dataset.session_target[0], delimiter=' ')
condition_mask=np.ones(len(behavioural),dtype=bool)
fmri_niimgs = index_img(fmri_filename,condition_mask)
session_label = behavioural['chunks']
masker = NiftiMasker(mask_img=mask_filename, sessions=session_label,
                     smoothing_fwhm=4, standardize=True,
                     memory="nilearn_cache", memory_level=1)
#Transform data so it is usable
fmri_masked = masker.fit_transform(fmri_niimgs)
session_label = OneHotEncoder(handle_unknown='ignore').fit_transform(session_label[:,None])
session_label = np.hstack((session_label.toarray(),np.zeros((session_label.shape[0],12))))
session_label = session_label + 0.00001*np.random.normal(size=(session_label.shape))
HX_tr,HX_te,HY_tr,HY_te=train_test_split(fmri_masked,session_label)


Dataset created in /root/nilearn_data/haxby2001



 ...done. (0 seconds, 0 min)


 ...done. (0 seconds, 0 min)


Downloaded 275546112 of 291168628 bytes (94.6%,    0.7s remaining) ...done. (13 seconds, 0 min)
Extracting data from /root/nilearn_data/haxby2001/f33ff337e914bf7fded743c7107979f9/subj2-2010.01.14.tar.gz..... done.
/usr/local/lib/python3.7/dist-packages/nilearn/_utils/helpers.py:145: FutureWarning: The parameter "sessions" will be removed in 0.9.0 release of Nilearn. Please use the parameter "runs" instead.
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/nilearn/input_data/nifti_masker.py:529: UserWarning: Persisting input arguments took 1.58s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  dtype=self.dtype
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: 

### Regularised CCA <a name="rccahaxby"></a>

In [10]:
# @markdown Execute this cell to change model regularisation (there's a bit of a lag as the model needs to fit in the background!)
style = {'description_width': 'initial'}
cx=widgets.FloatLogSlider(base=10,value=-1, min=-10, max=0,description='cx',readout=True,readout_format='.9f',style=style,continuous_update=False)
cy=widgets.FloatLogSlider(base=10,value=-1, min=-10, max=0,description='cy',readout=True,readout_format='.9f',style=style,continuous_update=False)
def interactive_cca(cx,cy):
    rcca=rCCA(latent_dims=1,c=[cx,cy]).fit([HX_tr,HY_tr])
    test_scores=rcca.transform([HX_te,HY_te])
    plot_latent_train_test(rcca.scores,test_scores)
    plot_train_test_corrs(rcca.scores,test_scores)
    plot_model_weights(rcca.weights[0],rcca.weights[1], title='Model weights')

plot_widget=widgets.interactive(interactive_cca, cx=cx,cy=cy)
display(plot_widget)

interactive(children=(FloatLogSlider(value=1e-10, continuous_update=False, description='cx', max=0.0, min=-10.…

### Sparse PLS <a name="splshaxby"></a>

In [11]:
# @markdown Execute this cell to change model regularisation
style = {'description_width': 'initial'}
c1=widgets.FloatSlider(value=3, min=1, max=np.sqrt(HX_tr.shape[1]),description='c1',readout=True,readout_format='.5f',style=style,continuous_update=False)
c2=widgets.FloatSlider(value=3, min=1, max=np.sqrt(HY_tr.shape[1]),description='c2',readout=True,readout_format='.5f',style=style,continuous_update=False)
def interactive_cca(c1,c2):
    spls=PMD(latent_dims=1,c=[c1,c2]).fit([HX_tr,HY_tr])
    test_scores=spls.transform([HX_te,HY_te])
    plot_latent_train_test(spls.scores,test_scores)
    plot_train_test_corrs(spls.scores,test_scores)
    plot_model_weights(spls.weights[0],spls.weights[1], title='Model weights')

plot_widget=widgets.interactive(interactive_cca, c1=c1,c2=c2)
display(plot_widget)

interactive(children=(FloatSlider(value=3.0, continuous_update=False, description='c1', max=21.540659228538015…

### Sparse CCA <a name="sccahaxby"></a>

In [12]:
# @markdown Execute this cell to change model regularisation
style = {'description_width': 'initial'}
c1=widgets.FloatLogSlider(value=0.005,base=10,min=-10, max=0,description='c1',readout=True,readout_format='.9f',style=style,continuous_update=False)
c2=widgets.FloatLogSlider(value=0.005,base=10,min=-10, max=0,description='c2',readout=True,readout_format='.9f',style=style,continuous_update=False)
def interactive_cca(c1,c2):
    scca=SCCA(latent_dims=1,c=0.01,initialization='random').fit([HX_tr,HY_tr])
    test_scores=scca.transform([HX_te,HY_te])
    plot_latent_train_test(scca.scores,test_scores)
    plot_train_test_corrs(scca.scores,test_scores)
    plot_model_weights(scca.weights[0],scca.weights[1], title='Model weights')
    
plot_widget=widgets.interactive(interactive_cca, c1=c1,c2=c2)
display(plot_widget)

interactive(children=(FloatLogSlider(value=0.005, continuous_update=False, description='c1', max=0.0, min=-10.…

## Conclusion <a name="conclusion"></a>
We have introduced Canonical Correlation Analysis and Partial Least Squares in an interactive way.

We showed how regularisation can be used to improve the generalizability and interpretability of our models.

We applied these models to a well-known neuroimaging dataset.

We introduced cca-zoo, an open-source python package that implements variants of CCA and PLS models with different regularisation effects. The goal of the package is to give researchers access to flexible regularisation methods. 

Further documentation is available at https://cca-zoo.readthedocs.io/en/latest/index.html 

the source code is available at https://github.com/jameschapman19/cca_zoo


### Acknowledgements
Thanks to the authors of the winners of last year's MEC 'Introduction to Medical Image Registration with DeepReg, Between Old and New'[8] for the basic structure of a tutorial notebook. Also thanks to the Neuromatch Academy summer school [9] who's extensive use of notebooks with widgets taught me everything I know!


## References <a name="references"></a>

[1] Helmer, Markus, et al. "On stability of Canonical Correlation Analysis and Partial Least Squares with application to brain-behavior associations." BioRxiv (2021): 2020-08.

[2] Vinod, Hrishikesh D. "Canonical ridge and econometrics of joint production." Journal of econometrics 4.2 (1976): 147-166.

[3] Witten, Daniela M., Robert Tibshirani, and Trevor Hastie. "A penalized matrix decomposition, with applications to sparse principal components and canonical correlation analysis." Biostatistics 10.3 (2009): 515-534.

[4] Mai, Qing, and Xin Zhang. "An iterative penalized least squares approach to sparse canonical correlation analysis." Biometrics 75.3 (2019): 734-744.

[5] Abraham, Alexandre, et al. "Machine learning for neuroimaging with scikit-learn." Frontiers in neuroinformatics 8 (2014): 14.

[6] Pedregosa, Fabian, et al. "Scikit-learn: Machine learning in Python." the Journal of machine Learning research 12 (2011): 2825-2830.

[7] Tibshirani, Robert. "Regression shrinkage and selection via the lasso." Journal of the Royal Statistical Society: Series B (Methodological) 58.1 (1996): 267-288.

[8] Fu, Yunguan, et al. "DeepReg: a deep learning toolkit for medical image registration." arXiv preprint arXiv:2011.02580 (2020).

[9] van Viegen, Tara, et al. "Neuromatch Academy: teaching computational neuroscience with global accessibility." arXiv preprint arXiv:2012.08973 (2020).